In [1]:
import torch
import gpytorch as gp
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score

torch.manual_seed(1)
np.random.seed(1)

In [2]:
df = pd.read_csv('C:/Users/tln229/Downloads/Python/1. Building/data/HVAC_B90_106_exp_30m_20221130.csv')

In [3]:
class MyGPModel(gp.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(MyGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module  = gp.means.ZeroMean()
        self.covar_module = gp.kernels.ScaleKernel(gp.kernels.RBFKernel())

    def forward(self, x):
        mean_x  = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gp.distributions.MultivariateNormal(mean_x, covar_x)

In [4]:
df_result = pd.DataFrame({'n_train':[], 'lr':[], 'R2':[]})
n_end = 3000

for n_train in ([32, 64, 128]):
    for lr in ([0.001, 0.01, 0.02, 0.05]):

        # TRAIN SET
        Tz_k   = np.array(df['room_temp']).reshape(-1,1)[0: n_train]
        Tz_k1  = np.array(df['room_temp'])[1: n_train+1]
        Tsa_k1 = np.array(df['supply_discharge_temp']).reshape(-1,1)[1: n_train+1]
        msa_k1 = np.array(df['airflow_current']).reshape(-1,1)[1: n_train+1]
        z      = ((msa_k1-400)/600)*(Tsa_k1-Tz_k)
        To_k   = np.array(df['thermostat_outside_temp']).reshape(-1,1)[0: n_train]

        train_x = np.concatenate((Tz_k, Tsa_k1, z, To_k), axis=1)
        train_x = torch.tensor(train_x, dtype=torch.float32)
        train_y = torch.tensor(Tz_k1, dtype=torch.float32)

        # TEST SET
        Tz_k   = np.array(df['room_temp']).reshape(-1,1)[n_train:n_end]
        Tz_k1  = np.array(df['room_temp'])[n_train+1:n_end+1]
        Tsa_k1 = np.array(df['supply_discharge_temp']).reshape(-1,1)[n_train+1: n_end+1]
        msa_k1 = np.array(df['airflow_current']).reshape(-1,1)[n_train+1:n_end+1]
        z      = ((msa_k1-400)/600)*(Tsa_k1-Tz_k)
        To_k   = np.array(df['thermostat_outside_temp']).reshape(-1,1)[n_train:n_end]

        test_x = np.concatenate((Tz_k, Tsa_k1, z, To_k), axis=1)
        test_x = torch.tensor(test_x, dtype=torch.float32)
        test_y = torch.tensor(Tz_k1, dtype=torch.float32)

        # initialize likelihood and model
        likelihood = gp.likelihoods.GaussianLikelihood()
        model      = MyGPModel(train_x, train_y, likelihood)

        # Training
        training_iter = 200000    # number of training iteration

        optimizer = torch.optim.Adam(model.parameters(), lr=lr)       # optimizer
        mll = gp.mlls.ExactMarginalLogLikelihood(likelihood, model)   # marginal likelihood

        R2_test = np.array([], dtype=np.float32)
        for i in range(training_iter+1):
            # TRAIN
            model.train()                   # find the hyperparameters
            likelihood.train()

            optimizer.zero_grad()           # Zero gradients from previous iteration
            output = model(train_x)         # Output from model
            loss   = -mll(output, train_y)  # Calc loss and backprop gradients

            loss.backward()
            optimizer.step()
            
            # EVAL
            model.eval()
            likelihood.eval()

            with torch.no_grad(), gp.settings.fast_pred_var():
                pred_y = likelihood(model(test_x))

            r2 = r2_score(test_y.numpy(), pred_y.mean.numpy())
            R2_test = np.append(R2_test, r2)
        
        df_result.loc[len(df_result)] = [n_train, lr, np.max(R2_test)]

with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 4,
                       ):
    print(df_result)